In [1]:
# basic functions and subject lists
# Read subject list, age list, and Jacobian Deter. of 3 study groups by sub_list 
import nipype.interfaces.io as nio
import os
# read data: 3 study groups by sub_list 
GROUPS=['PD','ET','NC']
OUT_DIR='/output/PD_ICA/'
SUB_LIST=[]; AGE_LIST=[]; JCOB_LIST=[];
print('Local data: ')
for group_name in GROUPS:
    current_group=group_name
    current_sub_list_file = '/codes/devel/PD_Marker/'+current_group+'_info_ICA.list'
    # create dir for output
    current_OUT_DIR=OUT_DIR+current_group+'/'
    if not os.path.exists(current_OUT_DIR):
        os.makedirs(current_OUT_DIR)
    #read sub list
    with open(current_sub_list_file, 'r') as f_sub:
        sub_list_raw= f_sub.readlines()
    sub_list = [x[0:-1].split('\t')[0] for x in sub_list_raw] # remove 
    age_list = [int(x[0:-1].split('\t')[1]) for x in sub_list_raw]
    SUB_LIST.append(sub_list);  AGE_LIST.append(age_list);
    N_sub=len(sub_list)
    print(group_name, ': ', N_sub)
    # grab group Jacobians
    ds_jacobian = nio.DataGrabber(infields=['sub_id'])
    ds_jacobian.inputs.base_directory = current_OUT_DIR # database
    ds_jacobian.inputs.template = '%s_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz' # from cwd
    ds_jacobian.inputs.sort_filelist = True
    ds_jacobian.inputs.sub_id = sub_list
    res_jacobian = ds_jacobian.run()
    jacobian_list=res_jacobian.outputs.outfiles
    JCOB_LIST.append(jacobian_list)
pd_sub_list = SUB_LIST[0]; et_sub_list = SUB_LIST[1]; nc_sub_list = SUB_LIST[2]; 
pd_age_list = AGE_LIST[0]; et_age_list = AGE_LIST[1]; nc_age_list = AGE_LIST[2];
pd_jaco_list=JCOB_LIST[0]; et_jaco_list=JCOB_LIST[1]; nc_jaco_list=JCOB_LIST[2];

from nipype.interfaces.ants import ANTS
import os,time
OUT_DIR='/output/PD_ICA/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)
atlas_09_masked='/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz'

# read group subject images given template
def dataGraber_sub(SUB_ID, DATA_DIR, TMPT_STR):
    import nipype.interfaces.io as nio
    import time
    t0=time.time()
    print('Grabbing files for: ', SUB_ID)
    OUT_FILE=[]
    out_len=len(TMPT_STR)
    if out_len == 0:
        print(SUB_ID+' has no files named: ', TMPT_STR)
        return OUT_FILE
    else:
        for i in range(out_len):
            TMP='%s/anat/%s_'+TMPT_STR[i]
            ds = nio.DataGrabber(infields=['subject_id', 'subject_id'])
            ds.inputs.base_directory = DATA_DIR # database
            ds.inputs.template = TMP 
            ds.inputs.subject_id = [SUB_ID]
            ds.inputs.sort_filelist = True
            res = ds.run()
            res_list = res.outputs.outfiles
            OUT_FILE.append(res_list)
        #print(SUB_ID+' files: ', OUT_FILE)
        print('dataGraber takes: ', time.time()-t0 )
        return OUT_FILE
    
# read subject affince and deformation fields from fMRIPrep results
def h5toWarp_nii(H5_FILE, OUT_DIR):
    from nipype.interfaces.ants import CompositeTransformUtil
    import os
    import time
    t0=time.time()
    tran = CompositeTransformUtil()
    tran.inputs.process = 'disassemble'
    tran.inputs.in_file = H5_FILE
    tran.inputs.out_file = OUT_DIR #bug
    #print(tran.cmdline)
    res=tran.run()
    out_warp  =OUT_DIR+'_Warp.nii.gz'
    out_affine=OUT_DIR+'_Affine.txt'
    os.system('mv '+res.outputs.displacement_field+' '+out_warp )
    os.system('mv '+res.outputs.affine_transform+' '+ out_affine)
    print('.h5 file disassemble takes: ', time.time()-t0 )
    return [out_warp, out_affine]

# Get masked subject Jacobians
def GetJacobian_nii(IN_IMAGE, MASK, JACOB_IMG, JACOB_MASK_IMG):
    # Jacobian of deformation field
    from nipype.interfaces.ants import CreateJacobianDeterminantImage
    from nipype.interfaces import fsl
    import time
    t0=time.time()
    jacobian = CreateJacobianDeterminantImage()
    jacobian.inputs.imageDimension = 3
    jacobian.inputs.deformationField = IN_IMAGE
    jacobian.inputs.outputImage = JACOB_IMG
    jacobian.inputs.num_threads = 4
    #print(jacobian.cmdline)
    jacobian.run()
    mask = fsl.ApplyMask(
        in_file=JACOB_IMG,
        out_file=JACOB_MASK_IMG,
        mask_file=MASK)
    mask.run()
    print('Jacobian takes: ', time.time()-t0 )
    return JACOB_MASK_IMG

201004-00:50:18,587 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0
Local data: 
PD :  40
ET :  29
NC :  33


In [2]:
pd_jaco_list

['/output/PD_ICA/PD/sub-0002_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',
 '/output/PD_ICA/PD/sub-0004_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',
 '/output/PD_ICA/PD/sub-0005_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',
 '/output/PD_ICA/PD/sub-0006_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',
 '/output/PD_ICA/PD/sub-0008_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',
 '/output/PD_ICA/PD/sub-0009_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',
 '/output/PD_ICA/PD/sub-0012_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',
 '/output/PD_ICA/PD/sub-0014_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',
 '/output/PD_ICA/PD/sub-0015_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',
 '/output/PD_ICA/PD/sub-0021_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',
 '/output/PD_ICA/PD/sub-0022_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz',

In [5]:
# MELODIC ICA (local pd+nc Jacobian masked)
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage

t0=time.time()
ICA_LOCAL_PATH ='/output/PD_ICA/ICA_local_pd+nc'
if not os.path.exists(ICA_LOCAL_PATH):
    os.makedirs(ICA_LOCAL_PATH)
dir_all_jacobian = pd_jaco_list+nc_jaco_list
merged_file = '/output/PD_ICA/local_pd+nc_4d.nii.gz'
#merger = fsl.Merge()
#merger.inputs.in_files = dir_all_jacobian
#merger.inputs.dimension = 'a'
#merger.inputs.merged_file = merged_file
#merger.cmdline
#merger.run()
t1=time.time()
print('Masked Jacobian:\n \tMegring 4D file takes: ', t1-t0)
melodic_setup = fsl.MELODIC()
melodic_setup.inputs.approach = 'tica'
melodic_setup.inputs.in_files = [merged_file]
melodic_setup.inputs.no_bet = True
melodic_setup.inputs.out_all = True
#melodic_setup.inputs.num_ICs = 30
melodic_setup.inputs.out_pca = True
melodic_setup.inputs.out_dir = ICA_LOCAL_PATH
melodic_setup.inputs.report = True
melodic_setup.cmdline
melodic_setup.run()
t2=time.time()
print('\tMELODIC ICA takes: ', t2-t1)

Masked Jacobian:
 	Megring 4D file takes:  0.0043773651123046875
	MELODIC ICA takes:  3227.0700254440308


In [4]:
# MELODIC ICA (local et+nc Jacobian masked)
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage

t0=time.time()
ICA_LOCAL_PATH ='/output/PD_ICA/ICA_local_et+nc'
if not os.path.exists(ICA_LOCAL_PATH):
    os.makedirs(ICA_LOCAL_PATH)
dir_all_jacobian = et_jaco_list+nc_jaco_list
merged_file = '/output/PD_ICA/local_et+nc_4d.nii.gz'
merger = fsl.Merge()
merger.inputs.in_files = dir_all_jacobian
merger.inputs.dimension = 'a'
merger.inputs.merged_file = merged_file
merger.cmdline
merger.run()
t1=time.time()
print('Masked Jacobian:\n \tMegring 4D file takes: ', t1-t0)
melodic_setup = fsl.MELODIC()
melodic_setup.inputs.approach = 'tica'
melodic_setup.inputs.in_files = [merged_file]
melodic_setup.inputs.no_bet = True
melodic_setup.inputs.out_all = True
melodic_setup.inputs.out_pca = True
melodic_setup.inputs.out_dir = ICA_LOCAL_PATH
melodic_setup.inputs.report = True
melodic_setup.cmdline
melodic_setup.run()
t2=time.time()
print('\tMELODIC ICA (ET+NC) takes: ', t2-t1)

Masked Jacobian:
 	Megring 4D file takes:  162.96515130996704
	MELODIC ICA (ET+NC) takes:  2399.2409434318542
